In [1]:
import pandas as pd

keys = ['sandwich','length','cheese','bread','vegetable','sauce']
kor_keys = ['샌드위치','길이','치즈','빵','채소','소스']

for key, kor_key in zip(keys,kor_keys):
    exec("%s = tuple(pd.read_csv('%s.csv'))"%(key,kor_key))
    print('<'+kor_key+'>')
    exec("print(*%s,sep=', ')"%(key))
    print()

<샌드위치>
페퍼 치킨 슈니첼, 페퍼로니 피자 썹, 쉬림프, 로스트 치킨, 에그마요, K-바비큐, 로티세리 바비큐 치킨, 풀드 포크 바비큐, 이탈리안 비엠티, 이탈리안 BLT, BMT, 비엘티, 미트볼, 햄, 참치, 써브웨이 클럽, 터키, 베지, 스테이크 & 치즈, 스테이크 앤 치즈, 터키 베이컨 아보카도, 스파이시 이탈리안, 치킨 데리야끼

<길이>
15센치, 15센치미터, 15센티, 15센티미터, 6인치, 6inch, 15cm, 30cm, 30센치, 30센티, 30센치미터, 30센티미터, 풋롱

<치즈>
아메리칸, 슈레드, 모차렐라, 모짜렐라, 아메리칸 치즈, 슈레드 치츠, 모차렐라 치츠

<빵>
화이트, 하티, 파마산오레가노, 위트, 허니오트, 플랫, 파마산

<채소>
양상추, 토마토, 오이, 피망, 양파, 피클, 올리브, 할라피뇨

<소스>
랜치, 마요네즈, 스위트 어니언, 허니 머스타드, 스위트 칠리, 핫 칠리, 사우스웨스트 치폴레, 머스타드, 홀스래디쉬, 올리브 오일, 레드와인식초, 소금, 후추, 스모크 바비큐



In [2]:
menu = {}
for key in keys:
    exec("menu['%s'] = %s"%(key,key))

In [3]:
sample_text = '15센치 화이트 빵으로 비엘티에 아메리칸 치즈 올려서\
 피망하고 파프리카 빼고 랜치랑 스위트 칠리 뿌려서 주세요..'

In [4]:
sample_data = '/length;15센치/ /bread;화이트/ 빵으로 /sandwich;비엘티/에\
 /cheese;아메리칸 치즈/ 올려서 /vegetable;피망/하고 /vegetable;파프리카/ 빼고\
 /sauce;랜치/랑 /sauce;스위트 칠리/ 뿌려서 주세요.'

In [5]:
def has_coda(word):
    # 받침으로 끝나면 false, 아니면 true 리턴
    if is_hangul(word):
        return (ord(word[-1]) - 44032) % 28 == 0
    return True

def is_hangul(word):
    code = ord(word[-1])
    if 44032 <= code <= 55203:
        return True
    return False

In [6]:
cat = {key:[val] for key,val in zip(keys, kor_keys)}
cat['length']+=['크기','사이즈']
cat['vegetable'].append('야채')
cat

{'sandwich': ['샌드위치'],
 'length': ['길이', '크기', '사이즈'],
 'cheese': ['치즈'],
 'bread': ['빵'],
 'vegetable': ['채소', '야채'],
 'sauce': ['소스']}

In [7]:
data_with_one = []
pos_predicates = ['으로 해주세요.','으로 할게요.','으로 주세요.','으로 주이소.']
neg_predicate = ' 빼주세요.'
predicate ='이요'

for key,li in menu.items():
    for val in li:
        if key == 'vegetable':
            # "{채소} 빼주세요."
            data_with_one.append('/%s;%s/'%(key,val)+neg_predicate)
            # "{veg}는 {채소} 빼주세요."
            for veg in cat[key]:
                data_with_one.append(veg+'는'+'/%s;%s/'%(key,val)+neg_predicate)
            # "{채소}은(는) 빼주세요."
            if has_coda(val):
                data_with_one.append('/%s;%s/'%(key,val)+'는'+neg_predicate)
            else:
                data_with_one.append('/%s;%s/'%(key,val)+'은'+neg_predicate)
        else:
            for pos_predicate in pos_predicates:
                if has_coda(val):
                    pos_predicate = pos_predicate[1:]
                # "{무엇}(으)로 해주세요/할게요/주세요."
                data_with_one.append('/%s;%s/'%(key,val)+pos_predicate)
                # "{종류}은(는) {무엇}(으)로 해주세요/할게요/주세요."
                for x in cat[key]:
                    if has_coda(x):
                        data_with_one.append(x+'는 '+'/%s;%s/'%(key,val)+pos_predicate)
                    else:
                        data_with_one.append(x+'은 '+'/%s;%s/'%(key,val)+pos_predicate)
        temp = predicate
        if has_coda(val):
            temp = predicate[1:]
        # "{무엇}(이)요"
        data_with_one.append('/%s;%s/'%(key,val)+temp)
        

print(*data_with_one[:10], sep='\n')

/sandwich;페퍼 치킨 슈니첼/으로 해주세요.
샌드위치는 /sandwich;페퍼 치킨 슈니첼/으로 해주세요.
/sandwich;페퍼 치킨 슈니첼/으로 할게요.
샌드위치는 /sandwich;페퍼 치킨 슈니첼/으로 할게요.
/sandwich;페퍼 치킨 슈니첼/으로 주세요.
샌드위치는 /sandwich;페퍼 치킨 슈니첼/으로 주세요.
/sandwich;페퍼 치킨 슈니첼/으로 주이소.
샌드위치는 /sandwich;페퍼 치킨 슈니첼/으로 주이소.
/sandwich;페퍼 치킨 슈니첼/이요
/sandwich;페퍼로니 피자 썹/으로 해주세요.


In [20]:
len(data_with_one)

720

In [43]:
pre_oven_data = []
## 빵과 메뉴
# 파마산에 햄으로 주세요/할게요.
# 에그마요에 (빵은) 플렛으로 주세요.
pre_oven_bread_sandwich = []
for bread in menu['bread']:
    for sandwich in menu['sandwich']:
        predicate = '으로 주세요.'
        if has_coda(bread):
            predicate = '로 주세요.'
        order = '/sandwich;'+sandwich+'/에 '
        pre_oven_data.append(order+'/bread;'+bread+'/'+predicate)
        pre_oven_data.append(order+'빵은 /bread;'+bread+'/'+predicate)
        
        predicate = '으로 주세요.'
        if has_coda(sandwich):
            predicate = '로 주세요.'
        order = '/bread;'+bread+'/에 '+'/sandwich;'+sandwich+'/'+predicate
        pre_oven_data.append(order)

        pre_oven_bread_sandwich.append(order[:-5]+' 해주시고, ')

## 빵과 길이
# (빵은) 파마산 오레가노 15센치(요./로 (해)주세요.)
# (빵은) 15센치 파마산 오레가노로 해주세요

for bread in menu['bread']:
    for length in menu['length']:
        predicate = '으로 해주세요.'
        if has_coda(bread):
            predicate = '로 해주세요.'
        order = '/length;'+length+'/ '+'/bread;'+bread+'/'+predicate
        pre_oven_data.append('빵은 '+order)
        pre_oven_data.append(order)
        
        predicate = '으로 해주세요.'
        if has_coda(length):
            predicate = '로 해주세요.'
        order = '/bread;'+bread+'/ '+'/length;'+length+'/'+predicate
        pre_oven_data.append('빵은 '+order)
        pre_oven_data.append(order)
         
## 빵, 길이, 메뉴
# 파마산 15샌치에 햄으로 주세요.

for bread in menu['bread']:
    bread = '/bread;'+bread+'/'
    for length in menu['length']:
        length = '/length;'+length+'/'
        bread_infos = [bread, bread+' '+length, length+' '+bread]
        for bread_info in bread_infos:
            for sandwich in menu['sandwich']:
                predicate = '으로 주세요.'
                if has_coda(sandwich):
                    predicate = '로 주세요.'
                sandwich = '/sandwich;'+sandwich+'/'
                ### {무엇}에(다가) {무엇}으로 주세요.
                # (빵)(+길이), (메뉴)
                pre_oven_data.append(bread_info+'에다가 '+sandwich+predicate)
                ###빵은 {빵}으로 해주시고, {메뉴}로 주세요
                   


## 빵, 메뉴, 치즈
# 화이트 빵에 이탈리안 비엠티 해주시고, 치즈는 아메리칸 치즈로 할게요.
for cheese in menu['cheese']:
    predicate = '으로 할게요.'
    if has_coda(cheese):
        predicate = '로 할게요.'
    order = '치즈는 '+'/cheese;'+cheese+'/'+predicate
    for x in pre_oven_bread_sandwich:
        pre_oven_data.append(x+order)
    

## 빵, 길이, 메뉴, 치즈
# 15cm 화이트에 이탈리안 비엠티 해주시고, 치즈는 아메리칸 치즈로 할게요.
for cheese in menu['cheese']:
    predicate = '으로 할게요.'
    if has_coda(cheese):
        predicate = '로 할게요.'
    order = '치즈는 '+'/cheese;'+cheese+'/'+predicate
    for x in pre_oven_bread_sandwich:
        for length in menu['length']:
            pre_oven_data.append('/length;'+length+'/ '+x+order)




print(*pre_oven_data, sep='\n') 

/sandwich;페퍼 치킨 슈니첼/에 /bread;화이트/로 주세요.
/sandwich;페퍼 치킨 슈니첼/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 주세요.
/sandwich;페퍼로니 피자 썹/에 /bread;화이트/로 주세요.
/sandwich;페퍼로니 피자 썹/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;페퍼로니 피자 썹/으로 주세요.
/sandwich;쉬림프/에 /bread;화이트/로 주세요.
/sandwich;쉬림프/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;쉬림프/로 주세요.
/sandwich;로스트 치킨/에 /bread;화이트/로 주세요.
/sandwich;로스트 치킨/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;로스트 치킨/으로 주세요.
/sandwich;에그마요/에 /bread;화이트/로 주세요.
/sandwich;에그마요/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;에그마요/로 주세요.
/sandwich;K-바비큐/에 /bread;화이트/로 주세요.
/sandwich;K-바비큐/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;K-바비큐/로 주세요.
/sandwich;로티세리 바비큐 치킨/에 /bread;화이트/로 주세요.
/sandwich;로티세리 바비큐 치킨/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;로티세리 바비큐 치킨/으로 주세요.
/sandwich;풀드 포크 바비큐/에 /bread;화이트/로 주세요.
/sandwich;풀드 포크 바비큐/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;풀드 포크 바비큐/로 주세요.
/sandwich;이탈리안 비엠티/에 /bread;화이트/로 주세요.
/sandwich;이탈리안 비엠티/에 

/length;15센치/ /bread;하티/에다가 /sandwich;풀드 포크 바비큐/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;이탈리안 비엠티/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;이탈리안 BLT/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;BMT/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;비엘티/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;미트볼/으로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;햄/으로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;참치/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;써브웨이 클럽/으로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;터키/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;베지/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;스테이크 & 치즈/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;스테이크 앤 치즈/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;터키 베이컨 아보카도/로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;스파이시 이탈리안/으로 주세요.
/length;15센치/ /bread;하티/에다가 /sandwich;치킨 데리야끼/로 주세요.
/bread;하티/에다가 /sandwich;페퍼 치킨 슈니첼/으로 주세요.
/bread;하티/에다가 /sandwich;페퍼로니 피자 썹/으로 주세요.
/bread;하티/에다가 /sandwich;쉬림프/로 주세요.
/bread;하티/에다가 /sandwich;로스트 치킨/으로 주세요.
/bread;하티

/bread;위트/에다가 /sandwich;로티세리 바비큐 치킨/으로 주세요.
/bread;위트/에다가 /sandwich;풀드 포크 바비큐/로 주세요.
/bread;위트/에다가 /sandwich;이탈리안 비엠티/로 주세요.
/bread;위트/에다가 /sandwich;이탈리안 BLT/로 주세요.
/bread;위트/에다가 /sandwich;BMT/로 주세요.
/bread;위트/에다가 /sandwich;비엘티/로 주세요.
/bread;위트/에다가 /sandwich;미트볼/으로 주세요.
/bread;위트/에다가 /sandwich;햄/으로 주세요.
/bread;위트/에다가 /sandwich;참치/로 주세요.
/bread;위트/에다가 /sandwich;써브웨이 클럽/으로 주세요.
/bread;위트/에다가 /sandwich;터키/로 주세요.
/bread;위트/에다가 /sandwich;베지/로 주세요.
/bread;위트/에다가 /sandwich;스테이크 & 치즈/로 주세요.
/bread;위트/에다가 /sandwich;스테이크 앤 치즈/로 주세요.
/bread;위트/에다가 /sandwich;터키 베이컨 아보카도/로 주세요.
/bread;위트/에다가 /sandwich;스파이시 이탈리안/으로 주세요.
/bread;위트/에다가 /sandwich;치킨 데리야끼/로 주세요.
/bread;위트/ /length;30cm/에다가 /sandwich;페퍼 치킨 슈니첼/으로 주세요.
/bread;위트/ /length;30cm/에다가 /sandwich;페퍼로니 피자 썹/으로 주세요.
/bread;위트/ /length;30cm/에다가 /sandwich;쉬림프/로 주세요.
/bread;위트/ /length;30cm/에다가 /sandwich;로스트 치킨/으로 주세요.
/bread;위트/ /length;30cm/에다가 /sandwich;에그마요/로 주세요.
/bread;위트/ /length;30cm/에다가 /sandwich;K-바비큐/로 주세요.
/bread;위트/ /length;30cm/에다가 /san

/bread;플랫/에다가 /sandwich;비엘티/로 주세요.
/bread;플랫/에다가 /sandwich;미트볼/으로 주세요.
/bread;플랫/에다가 /sandwich;햄/으로 주세요.
/bread;플랫/에다가 /sandwich;참치/로 주세요.
/bread;플랫/에다가 /sandwich;써브웨이 클럽/으로 주세요.
/bread;플랫/에다가 /sandwich;터키/로 주세요.
/bread;플랫/에다가 /sandwich;베지/로 주세요.
/bread;플랫/에다가 /sandwich;스테이크 & 치즈/로 주세요.
/bread;플랫/에다가 /sandwich;스테이크 앤 치즈/로 주세요.
/bread;플랫/에다가 /sandwich;터키 베이컨 아보카도/로 주세요.
/bread;플랫/에다가 /sandwich;스파이시 이탈리안/으로 주세요.
/bread;플랫/에다가 /sandwich;치킨 데리야끼/로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;페퍼 치킨 슈니첼/으로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;페퍼로니 피자 썹/으로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;쉬림프/로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;로스트 치킨/으로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;에그마요/로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;K-바비큐/로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;로티세리 바비큐 치킨/으로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;풀드 포크 바비큐/로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;이탈리안 비엠티/로 주세요.
/bread;플랫/ /length;15cm/에다가 /sandwich;이탈리안 BLT/로 주세요.
/bread;플랫/ 

/bread;위트/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;로티세리 바비큐 치킨/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;풀드 포크 바비큐/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;이탈리안 비엠티/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;BMT/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;비엘티/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;미트볼/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;참치/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;터키/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;베지/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;스테이크 & 치즈/로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/bread;위트/에 /sandwich;스테이크 앤 치즈/로 해주시고, 치즈는 /chee

...

/length;30센치미터/ /bread;플랫/에 /sandwich;미트볼/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;30센티미터/ /bread;플랫/에 /sandwich;미트볼/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;풋롱/ /bread;플랫/에 /sandwich;미트볼/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;15센치/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;15센치미터/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;15센티/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;15센티미터/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;6인치/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;6inch/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;15cm/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;30cm/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;30센치/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;30센티/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라/로 할게요.
/length;30센치미터/ /bread;플랫/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;모차렐라

/length;30cm/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;30센치/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;30센티/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;30센치미터/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;30센티미터/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;풋롱/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;15센치/ /bread;플랫/에 /sandwich;로티세리 바비큐 치킨/으로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;15센치미터/ /bread;플랫/에 /sandwich;로티세리 바비큐 치킨/으로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;15센티/ /bread;플랫/에 /sandwich;로티세리 바비큐 치킨/으로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;15센티미터/ /bread;플랫/에 /sandwich;로티세리 바비큐 치킨/으로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;6인치/ /bread;플랫/에 /sandwich;로티세리 바비큐 치킨/으로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;6inch/ /bread;플랫/에 /sandwich;로티세리 바비큐 치킨/으로 해주시고, 치즈는 /cheese;모짜렐라/로 할게요.
/length;15cm/ /bread;플랫/에 /sandwich;로티세리 바비큐 치킨/으로 해주시고, 치즈는 /ch

/length;6inch/ /bread;플랫/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;15cm/ /bread;플랫/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;30cm/ /bread;플랫/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;30센치/ /bread;플랫/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;30센티/ /bread;플랫/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;30센치미터/ /bread;플랫/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;30센티미터/ /bread;플랫/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;풋롱/ /bread;플랫/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;15센치/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;15센치미터/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;15센티/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;15센티미터/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/length;6인치/ /bread;플랫/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;아메리칸 치즈/로 할게요.
/

/length;6인치/ /bread;플랫/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;6inch/ /bread;플랫/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;15cm/ /bread;플랫/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;30cm/ /bread;플랫/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;30센치/ /bread;플랫/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;30센티/ /bread;플랫/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;30센치미터/ /bread;플랫/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;30센티미터/ /bread;플랫/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;풋롱/ /bread;플랫/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;15센치/ /bread;플랫/에 /sandwich;BMT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;15센치미터/ /bread;플랫/에 /sandwich;BMT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;15센티/ /bread;플랫/에 /sandwich;BMT/로 해주시고, 치즈는 /cheese;슈레드 치츠/로 할게요.
/length;15센티미터/ /bread;플랫/에 /sandwich;BMT/로 해주시고, 치즈는 /cheese;슈

/length;15센치미터/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;15센티/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;15센티미터/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;6인치/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;6inch/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;15cm/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;30cm/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;30센치/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;30센티/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;30센치미터/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;30센티미터/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;풋롱/ /bread;플랫/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /cheese;모차렐라 치츠/로 할게요.
/length;15센치/ /bread;플랫/에 /sandwic

In [44]:
len(pre_oven_data)

22904

In [25]:
from itertools import permutations

mult_veg_data = []
post_oven_veg = []
# 양파랑 피망 빼주세요.
# 채소/야채는 양파하고 피망 빼주세요.
# 양파랑 피망이랑 할라피뇨요.
# 양파, 피망, 할라피뇨요.
# 양파하고 피망하고 할라피뇨요.
# 양파 빼주시고 피망도 빼주세요.
# 양파, 피망 그리고 할라피뇨요.
connectives = ['이랑 ','하고 ',', ']
veges = menu['vegetable']
MAX_VEGE = 4
for i in range(2,MAX_VEGE+1):
    for p_veg in permutations(veges,i):
        order = ''
        for veg in p_veg:
            order+='/vegetable;'+veg+'/'+', '
        post_oven_veg.append(order[:-2] + ' 빼주시고, ')
        for p_con in permutations(connectives,i-1):
            order = ''
            for veg,con in zip(p_veg[:-1],p_con):
                order+='/vegetable;'+veg+'/'
                if con == '이랑 ':
                    if has_coda(veg):
                        con=con[1:]
                order+=con
            order+='/vegetable;'+p_veg[-1]+'/'
            # "{채소}(랑/하고/,) {채소}(랑/하고/,) {채소}요."
            order2=order
            if not has_coda(p_veg[-1]):
                order2+='이'
            mult_veg_data.append(order2+'요.')
            order+=' 빼주세요.'
            # "{채소}(랑/하고/,) {채소}(랑/하고/,) {채소} 빼주세요."
            mult_veg_data.append(order)
            for veg in cat['vegetable']:
                # "{veg}는 {채소}(랑/하고/,) {채소}(랑/하고/,) {채소} 빼주세요."
                mult_veg_data.append(veg+'는 '+order)
                
            order = ''
            for veg,con in zip(p_veg[:-2],p_con):
                order+='/vegetable;'+veg+'/'
                if con == '이랑 ':
                    if has_coda(veg):
                        con=con[1:]
                order+=con
            order+='/vegetable;'+p_veg[-2]+'/'+' 빼주시고 '
            order+='/vegetable;'+p_veg[-1]+'/'+'도 빼주세요.'
            # "{채소}(랑/하고/,) {채소} 빼주시고 {채소}도 빼주세요."
            mult_veg_data.append(order)
            for veg in cat['vegetable']:
                # "{veg}는 {채소}(랑/하고/,) {채소} 빼주시고 {채소}도 빼주세요."
                mult_veg_data.append(veg+'는 '+order)
                
        order=''
        for veg in p_veg[:-2]:
            order+='/vegetable;'+veg+'/'+', '
        order+='/vegetable;'+p_veg[-2]+'/'+' 그리고 '
        order+='/vegetable;'+p_veg[-1]+'/'+'요.'
        # "{채소}, {채소} 그리고 {채소}요."
        mult_veg_data.append(order)
            
    
print(*mult_veg_data[:25], sep='\n')                    

/vegetable;양상추/랑 /vegetable;토마토/요.
/vegetable;양상추/랑 /vegetable;토마토/ 빼주세요.
채소는 /vegetable;양상추/랑 /vegetable;토마토/ 빼주세요.
야채는 /vegetable;양상추/랑 /vegetable;토마토/ 빼주세요.
/vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
채소는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
야채는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
/vegetable;양상추/하고 /vegetable;토마토/요.
/vegetable;양상추/하고 /vegetable;토마토/ 빼주세요.
채소는 /vegetable;양상추/하고 /vegetable;토마토/ 빼주세요.
야채는 /vegetable;양상추/하고 /vegetable;토마토/ 빼주세요.
/vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
채소는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
야채는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
/vegetable;양상추/, /vegetable;토마토/요.
/vegetable;양상추/, /vegetable;토마토/ 빼주세요.
채소는 /vegetable;양상추/, /vegetable;토마토/ 빼주세요.
야채는 /vegetable;양상추/, /vegetable;토마토/ 빼주세요.
/vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
채소는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
야채는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
/vegetable;양상추/ 그리고 /vegetable;토마토/요.
/vegetable;양상추/랑 /vegetable;오이/요.
/vegetable;양상추/랑 /vegeta

In [26]:
len(mult_veg_data)

87920

In [27]:
mult_sauce_data = []
post_oven_sauce = []
connectives = ['이랑 ','하고 ',', ']
pos_predicates = ['으로 해주세요.','으로 할게요.','으로 주세요.','으로 주이소.']
sauces = menu['sauce']
MAX_SAUCE = 3
for i in range(2,MAX_SAUCE+1):
    for p_sauces in permutations(sauces,i):
        order = ''
        for sauce in p_sauces:
            order+='/sauce;'+sauce+'/'+', '
        post_oven_sauce.append(order[:-2] + ' 넣어주세요.')
        
        for p_con in permutations(connectives,i-1):
            order = ''
            for sauce,con in zip(p_sauces[:-1],p_con):
                order+='/sauce;'+sauce+'/'
                if con == '이랑 ':
                    if has_coda(sauce):
                        con=con[1:]
                order+=con
            order+='/sauce;'+p_sauces[-1]+'/'
            # "{채소}(랑/하고/,) {채소}(랑/하고/,) {채소}요."
            order2=order
            if not has_coda(p_sauces[-1]):
                order2+='이'
            mult_sauce_data.append(order2+'요.')
            
            for pos_predicate in pos_predicates:
                if has_coda(p_veg[-1]):
                    pos_predicate = pos_predicate[1:]
                mult_sauce_data.append(order+pos_predicate)
                
    
print(*mult_sauce_data[:25], sep='\n')                    

/sauce;랜치/랑 /sauce;마요네즈/요.
/sauce;랜치/랑 /sauce;마요네즈/로 해주세요.
/sauce;랜치/랑 /sauce;마요네즈/로 할게요.
/sauce;랜치/랑 /sauce;마요네즈/로 주세요.
/sauce;랜치/랑 /sauce;마요네즈/로 주이소.
/sauce;랜치/하고 /sauce;마요네즈/요.
/sauce;랜치/하고 /sauce;마요네즈/로 해주세요.
/sauce;랜치/하고 /sauce;마요네즈/로 할게요.
/sauce;랜치/하고 /sauce;마요네즈/로 주세요.
/sauce;랜치/하고 /sauce;마요네즈/로 주이소.
/sauce;랜치/, /sauce;마요네즈/요.
/sauce;랜치/, /sauce;마요네즈/로 해주세요.
/sauce;랜치/, /sauce;마요네즈/로 할게요.
/sauce;랜치/, /sauce;마요네즈/로 주세요.
/sauce;랜치/, /sauce;마요네즈/로 주이소.
/sauce;랜치/랑 /sauce;스위트 어니언/이요.
/sauce;랜치/랑 /sauce;스위트 어니언/로 해주세요.
/sauce;랜치/랑 /sauce;스위트 어니언/로 할게요.
/sauce;랜치/랑 /sauce;스위트 어니언/로 주세요.
/sauce;랜치/랑 /sauce;스위트 어니언/로 주이소.
/sauce;랜치/하고 /sauce;스위트 어니언/이요.
/sauce;랜치/하고 /sauce;스위트 어니언/로 해주세요.
/sauce;랜치/하고 /sauce;스위트 어니언/로 할게요.
/sauce;랜치/하고 /sauce;스위트 어니언/로 주세요.
/sauce;랜치/하고 /sauce;스위트 어니언/로 주이소.


In [28]:
len(mult_sauce_data)

68250

In [29]:
post_oven_data = []
for x in post_oven_veg:
    for y in post_oven_sauce:
        post_oven_data.append(x+y)
        

print(*post_oven_data[:25], sep='\n')                    

/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;마요네즈/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;스위트 어니언/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;허니 머스타드/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;스위트 칠리/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;핫 칠리/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;사우스웨스트 치폴레/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;머스타드/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;홀스래디쉬/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;올리브 오일/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;레드와인식초/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;소금/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;후추/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;스모크 바비큐/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;마요네즈/, /sauce

In [30]:
len(post_oven_data)

4902352

In [57]:
complex_data = []
## 완전 주문
# {빵} {길이} {샌드위치}에다가 치즈는 {치즈}로 해주시고, 채소는 {채소} 빼주시고,
# 소스는 {소스}로 해주세요.
MAX_VEG = 2
MAX_SAUCE = 2
for bread in menu['bread']:
    bread = '/bread;'+bread+'/ '
    for length in menu['length']:
        length = '/length;'+length+'/ '
        for sandwich in menu['sandwich']:
            sandwich = '/sandwich;'+sandwich+'/'
            for cheese in menu['cheese']:
                cheese_predicate = '으로 해주시고, '
                if has_coda(cheese):
                    cheese_predicate = '로 해주시고, '
                cheese = '/cheese;'+cheese+'/'
                for i in range(1,MAX_VEG+1):
                    for p_veg in permutations(menu['vegetable'],i):
                        vegs = ''
                        for veg in p_veg:
                            vegs+='/vegetable;'+veg+'/ '
                        for i in range(1,MAX_SAUCE+1):
                            for p_sauce in permutations(menu['sauce'],i):
                                sauces = ''
                                for sauce in p_sauce:
                                    sauces+='/sauce;'+sauce+'/ '
                                sauce_predicate = '으로 해주세요.'
                                if has_coda(sauce):
                                    sauce_predicate = '로 해주세요.'
                                complex_data.append(bread+length+sandwich+\
                                                    '에다가 치즈는 '+cheese+\
                                                   cheese_predicate+'채소는 '+\
                                                   vegs+' 빼주시고, '+\
                                                   sauces+sauce_predicate)
                        
                        

# {길이} 없을 때
# {치즈} 없을 때
# {채소} 없을 때
# {소스} 없을 때


print(*complex_data, sep='\n')                    

KeyboardInterrupt: 